In [1]:
# Fine-tuning LLM

In [2]:
! pip install transformers
! pip install dataset
! pip install sentencepiece
! pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.30
    Uninstalling SQLAlchemy-2.0.30:
      Successfully uninstalled SQLAlchemy-2.0.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

In [3]:
# Load a LLaMA2 model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# preprare sample data

In [6]:
sample_data = [
    {"input": "The capital of France is", "output": "Pairs"},
    {"input": "The tallest mountain in the world is", "output": "Mount Everest"}
]

In [7]:
# Generate more samples to total 100

In [11]:
for i in range(98):
  sample_data.append({"input": f"Sample input text {i}", "output": f"Sample output text {i}" })

In [13]:
# Convert sample data to a format suitable for fine-tuning
train_texts = [sample["input"] + " " + sample["output"] for sample in sample_data]

In [ ]:
train_texts

In [15]:
# Tokenize Data

In [26]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model.resize_token_embeddings(len(tokenizer))

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)

In [27]:
# Prepare Dataset for Fine-Tuning

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = TextDataset(train_encodings)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [29]:
# Fine-Tune the Model

In [36]:
from transformers import AdamW

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set the model in training mode
model.train()

# Training loop
for epoch in range(10):  # Number of epochs
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")


Epoch: 0, Loss: 2.367539405822754
Epoch: 0, Loss: 1.7704092264175415
Epoch: 0, Loss: 0.8058674335479736
Epoch: 0, Loss: 1.0801846981048584
Epoch: 0, Loss: 0.8340173363685608
Epoch: 0, Loss: 1.0729953050613403
Epoch: 0, Loss: 0.9648277759552002
Epoch: 0, Loss: 1.4708751440048218
Epoch: 0, Loss: 0.851088285446167
Epoch: 0, Loss: 1.0633578300476074
Epoch: 0, Loss: 0.6558752655982971
Epoch: 0, Loss: 0.781147301197052
Epoch: 0, Loss: 0.609009861946106
Epoch: 1, Loss: 1.4753180742263794
Epoch: 1, Loss: 0.7105396389961243
Epoch: 1, Loss: 1.1886485815048218
Epoch: 1, Loss: 0.7345221638679504
Epoch: 1, Loss: 1.0464305877685547
Epoch: 1, Loss: 0.8424046039581299
Epoch: 1, Loss: 0.7358371615409851
Epoch: 1, Loss: 0.7263699173927307
Epoch: 1, Loss: 0.7732903957366943
Epoch: 1, Loss: 1.0020477771759033
Epoch: 1, Loss: 0.6041343808174133
Epoch: 1, Loss: 1.169452428817749
Epoch: 1, Loss: 0.6309430599212646
Epoch: 2, Loss: 1.4377204179763794
Epoch: 2, Loss: 0.9142189025878906
Epoch: 2, Loss: 0.8869530

In [37]:
# Evaluate the Fine-Tuned Model
model.eval()

# Example input
input_text = "The capital of France is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [38]:
# Generate output
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=20)

output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The capital of France is Pairs 39 Sample output 39
